In [1]:
# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()
!pip install tensorflow


You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on

In [2]:
import numpy as np
import pandas as pd
import itertools

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import utils

# Use the Azure Machine Learning data preparation package
from azureml.dataprep import package

Using TensorFlow backend.


In [3]:
# This call will load the referenced package and return a DataFrame.
# If run in a PySpark environment, this call returns a
# Spark DataFrame. If not, it will return a Pandas DataFrame.
df = package.run('prep_airline_data.dprep', dataflow_idx=0)

# Remove this line and add code that uses the DataFrame
df.head(10)

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Month,Origin,TaxiIn,TaxiOut,UniqueCarrier,Year
0,128.0,116.0,-14.0,2211.0,4.0,3.0,8.0,2003.0,TPA,810.0,1.0,IAD,4.0,8.0,WN,2008.0
1,128.0,113.0,2.0,1002.0,4.0,3.0,19.0,754.0,TPA,810.0,1.0,IAD,5.0,10.0,WN,2008.0
2,96.0,76.0,14.0,804.0,4.0,3.0,8.0,628.0,BWI,515.0,1.0,IND,3.0,17.0,WN,2008.0
3,90.0,77.0,34.0,1959.0,4.0,3.0,34.0,1829.0,BWI,515.0,1.0,IND,3.0,10.0,WN,2008.0
4,101.0,87.0,11.0,2121.0,4.0,3.0,25.0,1940.0,JAX,688.0,1.0,IND,4.0,10.0,WN,2008.0
5,240.0,230.0,57.0,2037.0,4.0,3.0,67.0,1937.0,LAS,1591.0,1.0,IND,3.0,7.0,WN,2008.0
6,130.0,106.0,1.0,916.0,4.0,3.0,6.0,706.0,MCO,828.0,1.0,IND,5.0,19.0,WN,2008.0
7,121.0,107.0,80.0,1845.0,4.0,3.0,94.0,1644.0,MCO,828.0,1.0,IND,6.0,8.0,WN,2008.0
8,52.0,37.0,11.0,1021.0,4.0,3.0,9.0,1029.0,MDW,162.0,1.0,IND,6.0,9.0,WN,2008.0
9,228.0,213.0,15.0,1640.0,4.0,3.0,27.0,1452.0,PHX,1489.0,1.0,IND,7.0,8.0,WN,2008.0


In [9]:
train_size = int(len(df) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(df) - train_size))


Train size: 7975
Test size: 1994


In [10]:
#is it delayed?
#lets add a delayed flag if arrdelay > 15

df['delayed'] = np.where(df['ArrDelay']>=15, 1, 0)
len(df)

9969

In [11]:
#remove where cancelled
df = df.drop(df[df.Cancelled == 1.0].index)
len(df)

AttributeError: 'DataFrame' object has no attribute 'Cancelled'

In [12]:
data = df[df.columns[:-1]]
label = df["delayed"]


In [13]:
label.head(10)

0    0
1    0
2    0
3    1
4    0
5    1
6    0
7    1
8    0
9    1
Name: delayed, dtype: int64

In [14]:
train_data = data[:train_size]
train_label = label[:train_size]
test_data = data[train_size:]
test_label = label[train_size:]

In [15]:
#create hot-encode for airline, origin and destination

In [16]:
label_encoder = LabelEncoder()
origin = train_data.Origin
dest = train_data.Dest
#tailnum = train_data.TailNum
carrier = train_data.UniqueCarrier

train_data["origin_enc"] = label_encoder.fit_transform(origin)
train_data["dest_enc"] = label_encoder.fit_transform(dest)
#train_data["tailnum_enc"] = label_encoder.fit_transform(tailnum)
train_data["carrier_enc"] = label_encoder.fit_transform(carrier)

train_data = train_data.drop("Origin", 1)

train_data = train_data.drop("Dest", 1)

#train_data = train_data.drop("TailNum", 1)

train_data = train_data.drop("UniqueCarrier", 1)



In [17]:
origin = test_data.Origin
dest = test_data.Dest
#tailnum = test_data.TailNum
carrier = test_data.UniqueCarrier

test_data["origin_enc"] = label_encoder.fit_transform(origin)
test_data["dest_enc"] = label_encoder.fit_transform(dest)
#test_data["tailnum_enc"] = label_encoder.fit_transform(tailnum)
test_data["carrier_enc"] = label_encoder.fit_transform(carrier)

test_data = test_data.drop("Origin", 1)

test_data = test_data.drop("Dest", 1)

#test_data = test_data.drop("TailNum", 1)

test_data = test_data.drop("UniqueCarrier", 1)

In [18]:
train_data.shape

(7975, 16)

In [19]:
num_classes = np.max(train_label) + 1
train_label = utils.to_categorical(train_label, num_classes)
test_label = utils.to_categorical(test_label, num_classes)

In [22]:
#build model
# Build the model
# lets build a sequential model

batch_size = 32
epochs = 10

model = Sequential()

#first layer defines the shape of the input
model.add(Dense(512, input_shape=(16,)))

#then add a layer defining the activation function - relu is on of the most popular
#activation - should the neuron be fired or not - relu is quite efficient.

model.add(Activation('relu'))

# add a dropout layer. This is done to prevent overfitting - we discount a % of neurons
model.add(Dropout(0.5))

#here we define the output layer
#A dense layer is a classic fully connected neural network layer : each input node is connected to each output node.
model.add(Dense(2))

#for this layer use the softmax activation function
#softmax will assign a propability to each tag (total = 1)
model.add(Activation('softmax'))

#now we compile the model
#optimizer is the function used to search for the right weights
#here we are using Adaptive Moment Estimation (Adam) that also uses adaptive learning rates
#The loss function, also called the objective function is the evaluation of the model used by the optimizer to navigate the weight space.
#only accuracy is supported at the moment for metrics.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:

history = model.fit(train_data, train_label,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 7177 samples, validate on 798 samples
Epoch 1/10
7177/7177 [==============================] - 1s 185us/step - loss: 6.1722 - acc: 0.6167 - val_loss: 7.7157 - val_acc: 0.5213
Epoch 2/10
7177/7177 [==============================] - 1s 88us/step - loss: 6.1647 - acc: 0.6175 - val_loss: 7.7157 - val_acc: 0.5213
Epoch 3/10
7177/7177 [==============================] - 1s 86us/step - loss: 6.1647 - acc: 0.6175 - val_loss: 7.7157 - val_acc: 0.5213
Epoch 4/10
7177/7177 [==============================] - 1s 83us/step - loss: 6.1657 - acc: 0.6174 - val_loss: 7.7157 - val_acc: 0.5213
Epoch 5/10
7177/7177 [==============================] - 1s 88us/step - loss: 6.1647 - acc: 0.6175 - val_loss: 7.7157 - val_acc: 0.5213
Epoch 6/10
7177/7177 [==============================] - 1s 105us/step - loss: 6.1647 - acc: 0.6175 - val_loss: 7.7157 - val_acc: 0.5213
Epoch 7/10
7177/7177 [==============================] - 1s 95us/step - loss: 6.1647 - acc: 0.6175 - val_loss: 7.7157 - val_acc: 0.5213
Epoch 

In [24]:
score = model.evaluate(test_data, test_label,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

1994/1994 [==============================] - 0s 40us/step
Test score: 8.4874625689
Test accuracy: 0.473420260842


In [25]:
test_data.iloc[1]

ActualElapsedTime     112.0
AirTime               101.0
ArrDelay               14.0
ArrTime              1639.0
DayOfWeek               2.0
DayofMonth              8.0
DepDelay               22.0
DepTime              1347.0
Distance              816.0
Month                   1.0
TaxiIn                  4.0
TaxiOut                 7.0
Year                 2008.0
origin_enc             19.0
dest_enc               26.0
carrier_enc             0.0
Name: 7976, dtype: float64

In [26]:
for i in range(20):
    prediction = model.predict(np.array([test_data.iloc[i]]))
    predicted_label = np.argmax(prediction)
    print("predicted: %s actual: %s" % (predicted_label, np.argmax(test_label[i])))

predicted: 1 actual: 0
predicted: 1 actual: 0
predicted: 1 actual: 0
predicted: 1 actual: 1
predicted: 1 actual: 0
predicted: 1 actual: 0
predicted: 1 actual: 0
predicted: 1 actual: 1
predicted: 1 actual: 1
predicted: 1 actual: 0
predicted: 1 actual: 1
predicted: 1 actual: 1
predicted: 1 actual: 0
predicted: 1 actual: 1
predicted: 1 actual: 0
predicted: 1 actual: 1
predicted: 1 actual: 1
predicted: 1 actual: 1
predicted: 1 actual: 0
predicted: 1 actual: 0
